<a href="https://colab.research.google.com/github/tsshubhamv/teeth-whitening/blob/main/teeth_whitening_using_mediapipe_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Jai Shree Ganesh Ji

In [2]:
!pip install mediapipe
import mediapipe as mp
import cv2
import numpy as np
import math
from google.colab.patches import cv2_imshow
from google.colab import files
import requests
import urllib.request
import urllib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 43.6 MB/s eta 0:00:00


In [4]:
import cv2
import numpy as np
import mediapipe as mp
import urllib.request
import pandas as pd

def create_convex_polyfill(source_image, routes):
    # Create a blank image for drawing the polygon
    img = np.zeros_like(source_image, dtype='uint8')

    # Convert the routes to a numpy array
    points = np.array(routes, np.int32)

    # Reshape the array to a 2D shape
    points = points.reshape((-1, 1, 2))

    # Draw the convex polygon
    cv2.fillConvexPoly(img, points, color=(255, 255, 255))

    # Create a mask using the polygon
    mask = img

    # Create an inverse mask
    inverse_mask = cv2.bitwise_not(mask)

    return mask, inverse_mask


def change_lightness(img, value=30):
    # Convert the image to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    h, l, s = cv2.split(hls)

    # Increase the lightness channel by the given value
    l = cv2.add(l, value)
    l[l > 255] = 255
    l[l < 0] = 0

    # Merge the modified channels back to HLS image
    final_hls = cv2.merge((h, l, s))

    # Convert the image back to BGR color space
    img = cv2.cvtColor(final_hls, cv2.COLOR_HLS2BGR)

    return img


def change_saturation(img, value=30):
    # Convert the image to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    h, l, s = cv2.split(hls)

    # Increase the saturation channel by the given value
    s = cv2.add(s, value)
    s[s > 255] = 255
    s[s < 0] = 0

    # Merge the modified channels back to HLS image
    final_hls = cv2.merge((h, l, s))

    # Convert the image back to BGR color space
    img = cv2.cvtColor(final_hls, cv2.COLOR_HLS2BGR)

    return img


def get_red_colors(image):
    # Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper thresholds for red color in HSV
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])

    # Create masks for the red color ranges
    mask1 = cv2.inRange(hsv, lower_red, upper_red)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Combine the masks
    mask = cv2.bitwise_or(mask1, mask2)

    # Smoothen the mask using morphological opening
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    smoothed_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Remove the red color by setting pixels to black
    image_without_red = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(smoothed_mask))

    # Extract only the red color
    red_color = cv2.bitwise_and(image, image, mask=smoothed_mask)

    return image_without_red, red_color, smoothed_mask


def whiten_teeth_image_new(face_image_url):
    response = urllib.request.urlopen(face_image_url)
    image_data = response.read()
    image = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_UNCHANGED)
    img = image

    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    landmarks = results.multi_face_landmarks[0]

    inner_mouth = frozenset([
        (78, 191),
        (191, 80),
        (80, 81), (81, 82), (82, 13), (13, 312), (312, 311), (311, 310),
        (310, 415), (415, 308), (308, 324), (324, 318),
        (318, 402), (402, 317), (317, 14), (14, 87),
        (87, 178), (178, 88), (88, 95), (95, 78)
    ])
    df = pd.DataFrame(list(inner_mouth), columns=["p1", "p2"])

    routes_idx = []

    p1 = df.iloc[0]["p1"]
    p2 = df.iloc[0]["p2"]

    for i in range(0, df.shape[0]):
        obj = df[df["p1"] == p2]
        p1 = obj["p1"].values[0]
        p2 = obj["p2"].values[0]

        route_idx = []
        route_idx.append(p1)
        route_idx.append(p2)
        routes_idx.append(route_idx)

    routes = []
    for source_idx, target_idx in routes_idx:
        source = landmarks.landmark[source_idx]
        target = landmarks.landmark[target_idx]

        relative_source = (int(img.shape[1] * source.x), int(img.shape[0] * source.y))
        relative_target = (int(img.shape[1] * target.x), int(img.shape[0] * target.y))

        routes.append(relative_source)
        routes.append(relative_target)

    # Example usage
    source_image = img

    # Create convex polyfill and masks
    mask, inverse_mask = create_convex_polyfill(source_image, routes)

    # Apply masks to the source image
    masked_image = cv2.bitwise_and(source_image, mask)
    inverse_masked_image = cv2.bitwise_and(source_image, inverse_mask)

    # Removing red color
    image_without_red, red_color, red_mask = get_red_colors(masked_image)

    # Process the masked_image
    processed_masked_image = change_lightness(image_without_red, value=30)
    processed_masked_image = change_saturation(processed_masked_image, value=-15)

    cv2.imwrite("intm_image.jpg", processed_masked_image)

    # Apply the mask to the processed masked image
    processed_masked_image = cv2.bitwise_and(processed_masked_image, cv2.cvtColor(cv2.bitwise_not(red_mask), cv2.COLOR_GRAY2BGR))

    cv2.imwrite("intm_image_1.jpg", processed_masked_image)

    # Add red color back
    processed_masked_image = cv2.bitwise_or(processed_masked_image, red_color)

    processed_masked_image = cv2.bitwise_and(processed_masked_image, mask)

    # Combine the processed masked image with the inverse masked image
    output_image = cv2.bitwise_or(processed_masked_image, inverse_masked_image)

    # if CV2_DEBUG == True:
    cv2.imwrite('output_image_new.jpg', output_image)

    return output_image


In [5]:
CV2_DEBUG = True

In [10]:
whiten_teeth_image_new("https://img.buzzfeed.com/buzzfeed-static/static/2014-04/enhanced/webdr08/30/11/original-11354-1398871959-12.jpg")

array([[[238, 234, 239],
        [237, 233, 238],
        [236, 232, 237],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[237, 233, 238],
        [237, 233, 238],
        [236, 232, 237],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[237, 234, 236],
        [236, 233, 235],
        [236, 233, 235],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[ 15,  11,   6],
        [ 15,  11,   6],
        [ 16,  11,   8],
        ...,
        [  3,   4,   8],
        [  5,   4,   8],
        [  5,   4,   8]],

       [[ 12,   8,   3],
        [ 14,  10,   5],
        [ 15,  10,   7],
        ...,
        [  2,   3,   7],
        [  5,   4,   8],
        [  5,   4,   8]],

       [[ 11,   7,   2],
        [ 13,   9,   4],
        [ 15,  10,   7],
        ...,
        [  2,   3,   7],
        [  5,   4,   8],
        [  4,   3,   7]]